In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
# from lightgbm.sklearn import LGBMRegressor
# import lightgbm as lgb


# 코로나 데이터 처리

In [2]:
#코로나 데이터 읽어오기
covid = pd.read_csv('data_ver1/Covid19.csv')

In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

def grap_month_covid(data):
    data = str(data)
    return int(data[5:7])

def grap_date_covid(data):
    data = str(data)
    return int(data[8:10])

In [8]:
#데이터에서 불필요한 칼럼 제거 & 1,2,3,4월 데이터만 가져오기
covid = covid.drop(['time','released','deceased'],axis=1)
covid = covid[covid['date'].isin(['2020-01-31','2020-02-29','2020-03-31','2020-04-30'])]

#연월 형식 변경
covid['month'] = covid['date'].apply(lambda x: grap_month_covid(x))
covid['year'] = covid['date'].apply(lambda x: grap_year(x))
covid = covid.drop(['date'],axis=1)
covid = covid.sort_values(by=['province','year','month'])

be = ['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon',
       'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do']
af = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
       '충남', '전북', '전남', '경북', '경남', '제주']

# 지역 이름 한글로 변경
for i in range(len(covid['province'].unique())):
    covid['province'].replace(be[i],af[i], inplace=True)

In [9]:
covid_eda = covid.copy()

#누적된 코로나 확진자수 월별 확진자수로 변경
for i in range(len(covid)-1):
    if covid.iloc[i+1,0]==covid.iloc[i,0]:
        covid_eda.iloc[i+1,1]=covid.iloc[i+1,1]-covid.iloc[i,1]
        
covid_eda.rename({'province':'CARD_SIDO_NM'},axis='columns',inplace=True)

In [11]:
covid_eda

,CARD_SIDO_NM,confirmed,month,year
188,부산,0,1,2020
681,부산,80,2,2020
1208,부산,39,3,2020
1718,부산,18,4,2020
197,충북,0,1,2020
...,...,...,...,...
1717,서울,183,4,2020
193,울산,0,1,2020
686,울산,17,2,2020
1213,울산,22,3,2020


# 데이터프레임 통일작업(칼럼명, 칼럼값 형식 등)

In [ ]:
#데이터 불러오기
data_ = pd.read_csv('data_ver1/201901-202003.csv')

In [ ]:
#원본 데이터프레임 복사
bm = data.copy()

In [ ]:
#날짜 전처리
bm['month'] = bm['REG_YYMM'].apply(lambda x: grap_month(x))
bm['year'] = bm['REG_YYMM'].apply(lambda x: grap_year(x))
bm = bm.drop(['REG_YYMM'],axis=1)

In [ ]:
#필요없는 피처 제거
bm = bm.drop(['CARD_CCG_NM', 'HOM_CCG_NM','SEX_CTGO_CD','CSTMR_CNT','CNT'], axis=1)

In [ ]:
#10대20대, 나머지 세대 구분
bm['AGE']= bm['AGE'].apply(lambda x: '1020s' if x =='10s'or x=='20s' else '3060s')

In [ ]:
#독신, 나머지 구분
bm['FLC']= bm['FLC'].apply(lambda x: 'single' if x ==1 else 'family')

In [ ]:
#남은 피처 기준으로 groupby후 사용금액 더하기
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','year','FLC','month']
bm = bm.groupby(columns).sum().reset_index(drop=False)

In [ ]:
#관광지 구분(1,0 표시)
travel = ['제주','대구','전북','인천','부산','대전','광주','강원']
bm['out_home']= bm['CARD_SIDO_NM'].apply(lambda x: 1 if x in travel else 0)

In [ ]:
#카드 사용 지역과 거주지 동일여부로 데이터프레임 분리
home_condition = bm['CARD_SIDO_NM']==bm['HOM_SIDO_NM']
home = bm[home_condition]

out_condition = bm['CARD_SIDO_NM']!=bm['HOM_SIDO_NM']
out = bm[out_condition]